In [2]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt

#path = r"C://Users/Alok169231/Downloads/project_new/project_new-master/"

In [3]:
tempdf = pd.read_csv("loan_type_sec_unsec.csv") #loan type data file with secured and unsecured column

In [4]:
df = tempdf.copy()
df = df[df['cibil__user'].notnull()]
df['cibil__user'] = df['cibil__user'].astype(int).astype(object)

# overdue logic
date_in_past = dt.today() - datetime.timedelta(days=365*2)
df.closed_date = pd.to_datetime(df.closed_date)
df.last_payment_date = pd.to_datetime(df.last_payment_date)

def overdue(row, date_in_past): # date_in_past can be anytime in past i.e. 6, 12, 18, 24, 30, 36 months or even more.
    try:
        if pd.isnull(row.closed_date) or (row.closed_date >= date_in_past):
            if row.overdue !=0:
                return 1
    except:
        print("Exception Raised !")
    return 0

def loans_delayed(row, date_in_past): # date_in_past can be anytime in past i.e. 6, 12, 18, 24, 30, 36 months or even more.
    try:
        if row.last_payment_date>=date_in_past:
            if pd.isnull(row.closed_date)&(row.current_balance>0):
                if pd.isnull(row.last_payment_date)|((dt.today()-row.last_payment_date).days>=60): #60 -30 = DPD30
                    return 1
        else:
            return 0
    except:
        print("Exception Raised !")
    return 0

def group_data(df): # loan calculations based on past 2 years behaviour
    df['overdue_past'] = df.apply(lambda row: overdue(row, date_in_past), axis=1)
    df['loan_delayed'] = df.apply(lambda row: loans_delayed(row, date_in_past), axis=1)
    df = df.groupby('cibil__user').apply(lambda row: pd.Series(dict(
        overdue_amount_ongoing = row[row.overdue_past == 1].overdue.sum(),
        #overdue_number_ongoing = row[row.overdue_past == 1].overdue_past.sum(),
        #total_loans_ongoing = row[pd.isnull(row.closed_date)&(row.current_balance>0)].cibil__user.count(),
        total_emi_ongoing = row[pd.isnull(row.closed_date)].emi_amount.sum(),
        total_current_balance_ongoing = row[pd.isnull(row.closed_date)].current_balance.sum(),
        #num_settle_past = row[pd.isnull(row.closed_date)|(row.closed_date>=date_in_past)].settled.sum(),
        #num_loans_closed = row[pd.isnull(row.closed_date)|(row.closed_date>=date_in_past)].closed_date.count(),
        #num_loans_delayed = row[row.loan_delayed == 1].loan_delayed.count()
    )))
    return df

In [5]:
unsecdf = group_data(df[df['secured/unsecured'] == 'Unsecured'])
#secdf = group_data(df[df['secured/unsecured'] == 'Secured'])

C:\Users\ALOK16~1\AppData\Local\Temp/ipykernel_8864/113279571.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overdue_past'] = df.apply(lambda row: overdue(row, date_in_past), axis=1)
C:\Users\ALOK16~1\AppData\Local\Temp/ipykernel_8864/113279571.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loan_delayed'] = df.apply(lambda row: loans_delayed(row, date_in_past), axis=1)


In [6]:
unsecdf.columns = ["unsec_"+col for col in unsecdf.columns]
#secdf.columns = ["sec_"+col for col in secdf.columns]

In [11]:
#final_df = unsecdf.merge(secdf, on='cibil__user')

In [13]:
#final_df.to_csv("final_data_12-01-2022.csv")

In [14]:
unsecdf.to_csv("final_data_12-01-2022.csv")

,unsec_overdue_amount_ongoing,unsec_overdue_number_ongoing,unsec_total_loans_ongoing,unsec_total_emi_ongoing,unsec_total_current_balance_ongoing,unsec_num_settle_past,unsec_num_loans_closed,unsec_num_loans_delayed,sec_overdue_amount_ongoing,sec_overdue_number_ongoing,sec_total_loans_ongoing,sec_total_emi_ongoing,sec_total_current_balance_ongoing,sec_num_settle_past,sec_num_loans_closed,sec_num_loans_delayed
cibil__user,,,,,,,,,,,,,,,,
8,65570.0,4.0,4.0,6275.0,163619.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,36345.0,3.0,4.0,1397.0,172738.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,15864.0,2.0,11.0,9369.0,311405.0,1.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23,381319.0,7.0,20.0,29976.0,5506019.0,1.0,0.0,15.0,0.0,0.0,3.0,0.0,421000.0,0.0,1.0,0.0
46,72471.0,3.0,18.0,80612.0,8765072.0,0.0,1.0,10.0,0.0,0.0,2.0,0.0,4254745.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727093,0.0,0.0,2.0,3961.0,8048.0,0.0,0.0,0.0,814.0,1.0,1.0,-1.0,10259.0,0.0,0.0,0.0
1727150,0.0,0.0,2.0,-2.0,23187.0,0.0,0.0,0.0,-1.0,1.0,0.0,-1.0,-355874.0,0.0,0.0,0.0
1727223,0.0,0.0,1.0,-1.0,10572.0,0.0,0.0,1.0,0.0,0.0,1.0,3220.0,34495.0,0.0,0.0,1.0
